In [3]:
using Pkg; Pkg.activate("..")

  Activating project at `c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer`


In [4]:
using Flux, DLProteinFormats, Onion, RandomFeatureMaps, StatsBase, Plots

[ Info: Using local RoPE.jl (dev env)


In [5]:
dat = DLProteinFormats.load(PDBSimpleFlat500);

L = 30
train_inds = findall(dat.len .> L)
 
function random_batch(dat, L, B, filt_inds)
    locs = zeros(Float32, 3, L, B)
    inds = sample(filt_inds, B, replace=false)
    AAs = zeros(Int, L, B)
    for (i,ind) in enumerate(inds)
        l_range = rand(1:dat[ind].len - L + 1)
        locs[:, :, i] = dat[ind].locs[:, 1, l_range:l_range+L-1]
        AAs[:, i] = dat[ind].AAs[l_range:l_range+L-1]
    end
    return (;locs, AAs = Flux.onehotbatch(AAs, 1:20))
end

batch = random_batch(dat, L, 10, train_inds);
 

(locs = Float32[-1.3002242 -1.1566908 … 0.62550926 0.71040916; -0.7613289 -0.8056286 … -1.7723281 -1.6092618; 2.3987129 2.1252809 … 2.8664804 3.1106803;;; 0.6027733 0.3164402 … 0.7914734 0.6447735; 0.45070457 0.3482378 … 0.08787127 -0.01456213; -0.32564774 -0.32888108 … 0.71605223 0.964052;;; 1.2801039 1.4215373 … -0.46776256 -0.6848625; -0.5331842 -0.47061747 … -0.06838412 0.11204921; -1.0315678 -1.2856345 … 1.4772322 1.580299;;; 1.9724281 2.2653618 … -1.061238 -0.9377052; -0.05254669 0.03258667 … -0.19698028 -0.10661392; 0.21636124 0.3799612 … -1.4370722 -1.6989721;;; 1.9372044 2.1661048 … -0.20676212 -0.016295433; -0.23857078 -0.43613777 … -0.3800045 -0.5025711; -0.65939635 -0.6507961 … -1.4745295 -1.265763;;; 0.071567915 -0.12039852 … -0.28156528 -0.4357319; 1.063317 1.3041168 … 1.0247504 0.86725044; 3.474465 3.422398 … -0.7119687 -0.9248353;;; 1.4760973 1.4956639 … -0.7350361 -0.88353616; 0.30170518 0.554005 … -0.3175947 -0.53592813; -0.9225462 -1.1101129 … -0.59364617 -0.7488128;

In [6]:
struct Toy1{L}
    layers::L
end
Flux.@layer Toy1
function Toy1(dim, depth)
    layers = (;
        #loc_encoder = Dense(3 => dim, bias=false),
        transformers = [Onion.TransformerBlock(dim, 8, rope=Onion.MultiDimRoPE(8, 3)) for _ in 1:depth],
        #AA_decoder = Dense(dim => 20, bias=false),
    )
    return Toy1(layers)
end
function (m::Toy1)(x)
    l = m.layers
    for transformerblock in l.transformers
        x = transformerblock(x, 0, nothing, x_pos = x_pos)
        
    end
    return x 
end

In [ ]:
model = Toy1(64, 1)
opt_state = Flux.setup(AdamW(eta = 0.001), model)
losses = Float32[]

Float32[]

In [143]:
x =  randn(Float32, 64, 2, 1)
x_pos = randn(Float32, 3, 2, 1)

3×2×1 Array{Float32, 3}:
[:, :, 1] =
  1.71674    -0.432757
 -0.0165048   0.661345
 -0.34529     0.445952

In [144]:
model(x)

0.059496827


AssertionError: AssertionError: ("Output and shifted output not equal, mean differnce = ", nothing)

for epoch in 1:20 # 1:100
    tot_loss = 0f0
    for i in 1:1_000 # 1:10_000
        batch = random_batch(dat, L, 10, train_inds);
        l, grad = Flux.withgradient(model) do m
            aalogits = m(batch.locs)
            Flux.logitcrossentropy(aalogits, batch.AAs)
        end
        Flux.update!(opt_state, model, grad[1])
        tot_loss += l
        if mod(i, 50) == 0
            println(epoch, " ", i, " ", tot_loss/50)
            push!(losses, tot_loss/50)
            tot_loss = 0f0
        end
        (mod(i, 500) == 0) && savefig(plot(losses), "losses_toy_MultiDimRoPE.pdf")
    end
end
